## Doubly Linked List

In [111]:
class LinkedList:
    class Node:
        def __init__(self, value, prior=None, next=None):
            self.value = value
            self.prior = prior
            self.next = next
   

    def __init__(self):
        self.head = LinkedList.Node(None) # sentinel node
        self.head.prior = self.head.next = self.head # set up "circular" topology
        self.length = 0
  
    ### prepend and append, below, from class discussion
        
    def prepend(self, value):
        new_node = LinkedList.Node(value, prior=self.head, next=self.head.next)
        self.head.next.prior = self.head.next = new_node
        self.length += 1

        
    def append(self, value):
        new_node = LinkedList.Node(value, prior=self.head.prior, next=self.head)
        self.head.prior.next = self.head.prior = new_node
        self.length += 1 
        
            
    ### subscript-based access ###
    
    def _normalize_idx(self, idx):
        nidx = idx
        if nidx < 0:
            nidx += len(self)
            if nidx < 0:
                nidx = 0
        return nidx
    
    
    def __getitem__(self, idx):
        """
        Implements `x = self[idx]`
        """
        assert(isinstance(idx, int))
        idx = self._normalize_idx(idx)
        
        if idx > self.length - 1 or self.length == 0:
            raise IndexError
        elif idx == 0:
            return self.head.next.value
        elif idx == self.length - 1:
            return self.head.prior.value
        else:
            current_node = self.head.next
            while idx > 0:
                current_node = current_node.next
                idx -= 1
            return current_node.value
        

    def __setitem__(self, idx, value):
        """
        Implements `self[idx] = x`
        """
        assert(isinstance(idx, int))
        idx = self._normalize_idx(idx)
        
        if idx > self.length - 1 or self.length == 0:
            raise IndexError
        elif idx == 0:
            self.head.next.value = value
        elif idx == self.length - 1 or idx == -1:
            self.head.prior.value = value
        else:
            current_node = self.head.next
            while idx > 0:
                current_node = current_node.next
                idx -= 1
            current_node.value = value


    def __delitem__(self, idx):
        """
        Implements `del self[idx]`
        """
        assert(isinstance(idx, int))
        idx = self._normalize_idx(idx)
        
        if idx > self.length - 1 or self.length == 0:
            raise IndexError
        elif idx == 0 and self.length == 1:
            self.head.prior = self.head.next = self.head
        elif idx == self.length -1:
            self.head.prior.prior.next = self.head
            self.head.prior = self.head.prior.prior
        else:
            current_node = self.head.next
            while idx > 0:
                current_node = current_node.next
                idx -= 1
            current_node.prior.next = current_node.next
            current_node.next.prior = current_node.prior
        self.length -= 1
        

    ### stringification ###
    
    def __str__(self):
        """
        Implements `str(self)`. Returns '[]' if the list is empty, else
        returns `str(x)` for all values `x` in this list, separated by commas
        and enclosed by square brackets. E.g., for a list containing values
        1, 2 and 3, returns '[1, 2, 3]'.
        """
        if self.length == 0:
            return '[]'
        return '[' + ', '.join(str(e) for e in self.__iter__()) + ']'
                  
    def __repr__(self):
        """
        Supports REPL inspection. (Same behavior as `str`.)
        """
        if self.length == 0:
            return '[]'
        return '[' + ', '.join(str(e) for e in self.__iter__()) + ']'
    

    ### single-element manipulation ###
        
    def insert(self, idx, value):
        """
        Inserts value at position idx, shifting the original elements down the
        list, as needed. Note that inserting a value at len(self) --- equivalent
        to appending the value --- is permitted. Raises IndexError if idx is invalid.
        """
        assert(isinstance(idx, int))
        idx = self._normalize_idx(idx)
        
        if idx > self.length:
            raise IndexError
        elif idx == 0:
            self.prepend(value)
        elif idx == self.length:
            self.append(value)
        else:
            current_node = self.head.next
            while idx > 0:
                current_node = current_node.next
                idx -= 1
            new_node = LinkedList.Node(value, prior=current_node.prior, next=current_node)
            current_node.prior.next = new_node
            current_node.prior = new_node
            self.length += 1
        
    
    def pop(self, idx=-1):
        """
        Deletes and returns the element at idx (which is the last element,
        by default).
        """
        if idx > self.length - 1 or self.length == 0:
            raise ValueError
        else:
            value = None
            if idx == -1: # pop the last element
                value = sef.head.prior.value
                self.head.prior.prior.next = self.head
                self.head.prior = self.head.prior.prior
            else:
                idx = self._normalize_idx(idx)
                current_node = self.head.next
                while idx > 0:
                    current_node = current_node.next
                    idx -= 1
                value = current_node.value
                current_node.prior.next = current_node.next
                current_node.next.prior = current_node.prior 
            self.length -= 1
            return value
            
                   
    def remove(self, value):
        """
        Removes the first (closest to the front) instance of value from the
        list. Raises a ValueError if value is not found in the list.
        """
        current_node = self.head.next
        while current_node != self.head:
            if current_node.value == value:
                current_node.prior.next = current_node.next
                current_node.next.prior = current_node.prior
                self.length -= 1
                return
            current_node = current_node.next
        raise ValueError   
        
        
    ### predicates (T/F queries) ###
    
    def __eq__(self, other):
        """
        Returns True if this LinkedList contains the same elements (in order) as
        other. If other is not an LinkedList, returns False.
        """
        if not isinstance(other, LinkedList):
            return False
        elif self.length != other.length:
            return False
        else:
            self_node = self.head.next
            other_node = other.head.next
            while self_node != self.head:
                if self_node.value != other_node.value:
                    return False
                self_node = self_node.next
                other_node = other_node.next
            return True
                    
            
    def __contains__(self, value):
        """
        Implements `val in self`. Returns true if value is found in this list.
        """
        current_node = self.head.next
        while current_node != self.head:
            if current_node.value == value:
                return True
            current_node = current_node.next
        return False

    ### queries ###
    
    def __len__(self):
        """
        Implements `len(self)`
        """
        return self.length

    
    def min(self):
        """
        Returns the minimum value in this list.
        """
        if self.length == 0:
            raise ValueError
        elif self.length == 1:
            return self.head.next.value
        else:
            min_value = self.head.next.value
            current_node = self.head.next.next
            while current_node != self.head:
                if current_node.value < min_value:
                    min_value = current_node.value
                current_node = current_node.next
            return min_value
                
    
    def max(self):
        """
        Returns the maximum value in this list.
        """
        if self.length == 0:
            raise ValueError
        elif self.length == 1:
            return self.head.next.value
        else:
            max_value = self.head.next.value
            current_node = self.head.next.next
            while current_node != self.head:
                if current_node.value > max_value:
                    max_value = current_node.value
                current_node = current_node.next
            return max_value

    
    def index(self, value, i=0, j=None):
        """
        Returns the index of the first instance of value encountered in
        this list between index i (inclusive) and j (exclusive). If j is not
        specified, search through the end of the list for value. If value
        is not in the list, raise a ValueError.
        """         
        if not j:
            j = self.length
        else:
            j = self._normalize_idx(j)
        
        if self.length == 0:
            raise ValueError
        elif self.length == 1:
            if i < j and self.head.next.value == value:
                return 0
            else:
                raise ValueError
        else:
            i = self._normalize_idx(i)
            if j <= i:
                raise ValueError
            current_node = self.head.next
            index = 0 # keep track of i
            if i: # i != 0
                index = i
                while i > 0:
                    current_node = current_node.next
                    i -= 1
            while index < j:
                if current_node.value == value:
                    return index
                current_node = current_node.next
                index += 1
            raise ValueError
        
    
    def count(self, value):
        """
        Returns the number of times value appears in this list.
        """
        if self.length == 0:
            return 0
        elif self.length == 1:
            if self.head.next.value == value:
                return 1
            return 0
        else:
            count = 0
            current_node = self.head.next
            while current_node != self.head:
                if current_node.value == value:
                    count += 1
                current_node = current_node.next
            return count

    
    ### bulk operations ###

    def __add__(self, other):
        """
        Implements `self + other_list`. Returns a new LinkedList
        instance that contains the values in this list followed by those 
        of other.
        """
        assert(isinstance(other, LinkedList))
        new_list = LinkedList()
        new_node = new_list.head
        self_node = self.head.next
        while self_node != self.head:
            node = LinkedList.Node(self_node.value, prior = new_node, next=new_list.head)
            new_node.next = node
            new_list.head.prior = node
            self_node = self_node.next
            new_node = new_node.next
        other_node = other.head.next
        while other_node != other.head:
            node = LinkedList.Node(other_node.value, prior = new_node, next=new_list.head)
            new_node.next = node
            new_list.head.prior = node
            other_node = other_node.next
            new_node = new_node.next
        new_list.length = self.length + other.length
        self = other = None
        return new_list
        
          
    def clear(self):
        """
        Removes all elements from this list.
        """
        self.head.prior = self.head.next = self.head
        self.length = 0

        
    def copy(self):
        """
        Returns a new LinkedList instance (with separate Nodes), that
        contains the same values as this list.
        """
        new_list = LinkedList()
        new_node = new_list.head
        self_node = self.head.next
        while self_node != self.head:
            node = LinkedList.Node(self_node.value, prior=new_node, next=new_list.head)
            new_node.next = node
            new_list.head.prior = node
            self_node = self_node.next
            new_node = new_node.next
        new_list.length = self.length
        self = None
        return new_list


    def extend(self, other):
        """
        Adds all elements, in order, from other --- an Iterable --- to this list.
        """
        for node in other:
            self.append(node)
            
            
    ### iteration ###

    def __iter__(self):
        """
        Supports iteration (via `iter(self)`)
        """
        if self.length == 0:
            raise StopIteration
        self_node = self.head.next
        while self_node != self.head:
            yield self_node.value
            self_node = self_node.next

In [112]:
# test subscript-based access

from unittest import TestCase
import random

tc = TestCase()
data = [1, 2, 3, 4]
lst = LinkedList()
for d in data:
    lst.append(d)

for i in range(len(data)):
    tc.assertEqual(lst[i], data[i])
    
with tc.assertRaises(IndexError):
    x = lst[100]

with tc.assertRaises(IndexError):
    lst[100] = 0

with tc.assertRaises(IndexError):
    del lst[100]

lst[1] = data[1] = 20
del data[0]
del lst[0]

for i in range(len(data)):
    tc.assertEqual(lst[i], data[i])

data = [random.randint(1, 100) for _ in range(100)]
lst = LinkedList()
for d in data:
    lst.append(d)

for i in range(len(data)):
    lst[i] = data[i] = random.randint(101, 200)
for i in range(50):
    to_del = random.randrange(len(data))
    del lst[to_del]
    del data[to_del]

for i in range(len(data)):
    tc.assertEqual(lst[i], data[i])
    
for i in range(0, -len(data), -1):
    tc.assertEqual(lst[i], data[i])

In [113]:
# test stringification

from unittest import TestCase
tc = TestCase()

lst = LinkedList()
tc.assertEqual('[]', str(lst))
tc.assertEqual('[]', repr(lst))

lst.append(1)
tc.assertEqual('[1]', str(lst))
tc.assertEqual('[1]', repr(lst))

lst = LinkedList()
for d in (10, 20, 30, 40, 50):
    lst.append(d)
tc.assertEqual('[10, 20, 30, 40, 50]', str(lst))
tc.assertEqual('[10, 20, 30, 40, 50]', repr(lst))

In [114]:
# test single-element manipulation

from unittest import TestCase
import random

tc = TestCase()
lst = LinkedList()
data = []

for _ in range(100):
    to_ins = random.randrange(1000)
    ins_idx = random.randrange(len(data)+1)
    data.insert(ins_idx, to_ins)
    lst.insert(ins_idx, to_ins)

for i in range(100):
    tc.assertEqual(data[i], lst[i])

for _ in range(50):
    pop_idx = random.randrange(len(data))
    tc.assertEqual(data.pop(pop_idx), lst.pop(pop_idx))
    
for i in range(50):
    tc.assertEqual(data[i], lst[i])

for _ in range(25):
    to_rem = data[random.randrange(len(data))]
    data.remove(to_rem)
    lst.remove(to_rem)
    
for i in range(25):
    tc.assertEqual(data[i], lst[i])

with tc.assertRaises(ValueError):
    lst.remove(9999)

In [115]:
# test predicates
from unittest import TestCase

tc = TestCase()
lst = LinkedList()
lst2 = LinkedList()

tc.assertEqual(lst, lst2)

lst2.append(100)
tc.assertNotEqual(lst, lst2)

lst.append(100)
tc.assertEqual(lst, lst2)

tc.assertFalse(1 in lst)
tc.assertFalse(None in lst)

lst = LinkedList()
for i in range(100):
    lst.append(i)
tc.assertFalse(100 in lst)
tc.assertTrue(50 in lst)

In [116]:
# test queries

from unittest import TestCase
tc = TestCase()
lst = LinkedList()

tc.assertEqual(0, len(lst))
tc.assertEqual(0, lst.count(1))
with tc.assertRaises(ValueError):
    lst.index(1)

import random
data = [random.randrange(1000) for _ in range(100)]
for d in data:
    lst.append(d)

tc.assertEqual(100, len(lst))
tc.assertEqual(min(data), lst.min())
tc.assertEqual(max(data), lst.max())
for x in data:    
    tc.assertEqual(data.index(x), lst.index(x))
    tc.assertEqual(data.count(x), lst.count(x))

with tc.assertRaises(ValueError):
    lst.index(1000)

lst = LinkedList()
for d in (1, 2, 1, 2, 1, 1, 1, 2, 1):
    lst.append(d)
tc.assertEqual(1, lst.index(2))
tc.assertEqual(1, lst.index(2, 1))
tc.assertEqual(3, lst.index(2, 2))
tc.assertEqual(7, lst.index(2, 4))
tc.assertEqual(7, lst.index(2, 4, -1))
with tc.assertRaises(ValueError):
    lst.index(2, 4, -2)

In [117]:
# test bulk operations

from unittest import TestCase
tc = TestCase()
lst = LinkedList()
lst2 = LinkedList()
lst3 = lst + lst2

tc.assertIsInstance(lst3, LinkedList)
tc.assertEqual(0, len(lst3))

import random
data  = [random.randrange(1000) for _ in range(50)]
data2 = [random.randrange(1000) for _ in range(50)]
for d in data:
    lst.append(d)
for d in data2:
    lst2.append(d)
lst3 = lst + lst2
tc.assertEqual(100, len(lst3))
data3 = data + data2
for i in range(len(data3)):
    tc.assertEqual(data3[i], lst3[i])

lst.clear()
tc.assertEqual(0, len(lst))
with tc.assertRaises(IndexError):
    lst[0]

for d in data:
    lst.append(d)
lst2 = lst.copy()
tc.assertIsNot(lst, lst2)
tc.assertIsNot(lst.head.next, lst2.head.next)
for i in range(len(data)):
    tc.assertEqual(lst[i], lst2[i])
tc.assertEqual(lst, lst2)

lst.clear()
lst.extend(range(10))
lst.extend(range(10,0,-1))
lst.extend(data.copy())
tc.assertEqual(70, len(lst))

data = list(range(10)) + list(range(10, 0, -1)) + data
for i in range(len(data)):
    tc.assertEqual(data[i], lst[i])

In [118]:
# test iteration

from unittest import TestCase
tc = TestCase()
lst = LinkedList()

import random
data = [random.randrange(1000) for _ in range(100)]
lst = LinkedList()
for d in data:
    lst.append(d)
tc.assertEqual(data, [x for x in lst])

it1 = iter(lst)
it2 = iter(lst)
for x in data:
    tc.assertEqual(next(it1), x)
    tc.assertEqual(next(it2), x)